In [1]:
import torch
import matplotlib.pyplot as plt
import torch_dct as dct
from lib.tools import *
from lib.decomposition import *
from lib.jpeg import *
from lib.sketchml import *
from lib.regression import *
import os
import pandas as pd

In [2]:
# tensor_dir = "../experiments_fotball/tensors/"
tensor_dir = "./test_tensors/layer6/"
output_dir = "./measurements/"
layer = 6

test_tensors = os.listdir(tensor_dir)

In [3]:
## Measurements
split_layer = []

tensor_rank=[]
tensor_sparsity=[]
tensor_regularity=[]
tensor_pictoriality=[]
tensor_decomposability=[]

jpeg_ratio=[]
jpeg_snr=[]
jpeg_mse=[]
decomposition_ratio=[]
decomposition_snr=[]
decomposition_mse=[]
regression_ratio=[]
regression_snr=[]
regression_mse=[]
sketchml_ratio=[]
sketchml_snr=[]
sketchml_mse=[]

In [4]:
for tensor_name in test_tensors[0:10]:
    print("************* Testing tensor: "+tensor_name+"***************")
    split_layer.append(layer)
    tensor = torch.load(tensor_dir+tensor_name)
    # tensor = tensor * (abs(tensor)>0.001)
    tensor = tensor[0]

    # tensor_rank.append(calculate_cp_rank(tensor.numpy(),max_rank=10,tol=1e-4))
    tensor_rank.append(calculate_slice_avg_rank(tensor))
    tensor_sparsity.append(calculate_sparsity(tensor.numpy()))
    tensor_pictoriality.append(1-get_tensor_pictoriality(tensor)/(tensor.numel()*4))
    tensor_regularity.append(1-get_tensor_regularity(tensor)/(tensor.numel()*4))
    tensor_decomposability.append(1-((tensor_rank[-1]*tensor.shape[1]+tensor_rank[-1]*tensor.shape[2])*tensor.shape[0])/(tensor.numel()*4))
    print("Rank: %f, Sparsity: %f, Picoriality: %f, Regularity: %f, Decomposability: %f" % (tensor_rank[-1],tensor_sparsity[-1],tensor_pictoriality[-1],tensor_regularity[-1],tensor_decomposability[-1]))

    ## Jpeg compression
    print("Doing JPEG compression")
    normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg_gray(tensor, 100)
    reconstructed_tensor_jpeg = decompressor_jpeg_gray(tensor.shape,normalize_base, scale,zero_point, encoded_data )
    ratio_jpeg = tensor.numel()*4/compressed_size_jpeg
    snr_jpeg = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
    jpeg_ratio.append(ratio_jpeg)
    jpeg_snr.append(snr_jpeg)
    jpeg_mse.append(mse_jpeg)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_jpeg,snr_jpeg,mse_jpeg))

    print("Doing Decomposition compression")
    # factors, compressed_size_decomposition = compressor_decomposition(tensor, tensor_rank[-1])
    # reconstructed_tensor_decomposition = decompressor_decomposition(factors)
    factors, compressed_size_decomposition = compressor_decomposition_slice(tensor)
    reconstructed_tensor_decomposition = decompressor_decomposition_slice(tensor.shape,factors)
    snr_decomposition = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    mse_decomposition = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition.numpy())
    ratio_decomposition= tensor.numel()*4/compressed_size_decomposition
    decomposition_ratio.append(ratio_decomposition)
    decomposition_snr.append(snr_decomposition)
    decomposition_mse.append(mse_decomposition)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_decomposition,snr_decomposition,mse_decomposition))

    print("Doing Regression compression")
    factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 3)
    reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
    snr_regression = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_regression.numpy())
    mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
    ratio_regression = tensor.numel()*4 / compressed_size_regression
    regression_ratio.append(ratio_regression)
    regression_snr.append(snr_regression)
    regression_mse.append(mse_regression)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_regression,snr_regression,mse_regression))

    print("Doing SketchML compression")
    minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,256,0.3,3)
    reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
    snr_sketchml = calculate_snr(tensor.size(),tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
    ratio_sketchml = tensor.numel() *4 / compressed_size_sketchml
    sketchml_ratio.append(ratio_sketchml)
    sketchml_snr.append(snr_sketchml)
    sketchml_mse.append(mse_sketchml)
    print("Ratio: %f, SNR: %f, MSE: %f" % (ratio_sketchml,snr_sketchml,mse_sketchml))


************* Testing tensor: tensor_l6_98.pt***************
Rank: 50.890625, Sparsity: 0.000000, Picoriality: 0.997621, Regularity: 0.990324, Decomposability: 0.510667
Doing JPEG compression
Ratio: 6.413936, SNR: 29.177122, MSE: 0.000000
Doing Decomposition compression
Ratio: 0.510900, SNR: 112.465296, MSE: 0.000000
Doing Regression compression
Ratio: 15.720930, SNR: 6.460177, MSE: 0.000041
Doing SketchML compression


KeyboardInterrupt: 

In [5]:
dict = {'split_layer':split_layer,
        'rank': tensor_rank, 
        'sparsity': tensor_sparsity, 
        'regularity': tensor_regularity,
        'pictoriality': tensor_pictoriality,
        'decomposability':tensor_decomposability,
        'jpeg_ratio':jpeg_ratio,
        'jpeg_snr':jpeg_snr,
        'jpeg_mse':jpeg_mse,
        'decomposition_ratio':decomposition_ratio,
        'decomposition_snr':decomposition_snr,
        'decomposition_mse':decomposition_mse,
        'regression_ratio':regression_ratio,
        'regression_snr':regression_snr,
        'regression_mse':regression_mse,
        'sketchml_ratio':sketchml_ratio,
        'sketchml_snr':sketchml_snr,
        'sketchml_mse':sketchml_mse
        } 
   
df = pd.DataFrame(dict)

In [6]:
df.to_csv(output_dir+"layer"+str(layer)+".csv",index=False, header=True)

In [7]:
df.head()

,split_layer,rank,sparsity,regularity,pictoriality,decomposability,jpeg_ratio,jpeg_snr,jpeg_mse,decomposition_ratio,decomposition_snr,decomposition_mse,regression_ratio,regression_snr,regression_mse,sketchml_ratio,sketchml_snr,sketchml_mse
0,6,50.890625,0.0,0.990324,0.997621,0.510667,6.413936,29.177122,2.210146e-07,0.510900,112.465296,1.036581e-15,15.72093,6.460177,0.000041,4.348550,1.371853,0.000133
1,6,50.812500,0.0,0.989246,0.997038,0.511418,5.916343,28.212636,2.868829e-07,0.511685,109.522524,2.121856e-15,15.72093,6.282918,0.000045,4.349862,1.304457,0.000141
2,6,50.812500,0.0,0.989792,0.997589,0.511418,6.000659,27.923324,3.110791e-07,0.511685,110.225334,1.830921e-15,15.72093,6.102831,0.000047,4.348223,1.383225,0.000140
3,6,50.812500,0.0,0.988909,0.997516,0.511418,5.882557,28.773205,2.800005e-07,0.511685,109.579859,2.325375e-15,15.72093,5.989724,0.000053,4.352542,1.361898,0.000154
4,6,50.828125,0.0,0.989441,0.997430,0.511268,6.229069,29.574695,2.146454e-07,0.511528,109.398670,2.235238e-15,15.72093,6.280167,0.000046,4.351092,1.281643,0.000145
